# Обучение моделей с помощью Экспериментов Azure ML

## 1. Соединение со своим Workspace

Импорт модулей:

In [ ]:
import os

import azureml.core
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.core.datastore import Datastore
from azureml.train.sklearn import SKLearn

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

Создаем соединение со своей Рабоечей областью:

In [ ]:
ws = Workspace.from_config()
print('Connected Workspace:', ws.name, ws.resource_group, ws.location, sep = '\n')

## 2. Подготовка данных для Эксперимент

Зададим имя для эксперимента:

In [ ]:
experiment_name = 'diabetes-train-model-experiment'
experiment_dir = experiment_name

Воспользуемся данными загружеными на [прошлом лабораторном занятии](../module-3/azure-datastore.ipynb) и загрузим их в папку эксперимента:

In [ ]:
# Get uploaded data 
ds = ws.get_default_datastore()
diabetes_ds = Dataset.Tabular.from_delimited_files(path=(ds, 'diabetes-data/*.csv'))

data = diabetes_ds.to_pandas_dataframe()
data.to_csv(f'{experiment_name}/data.csv', index=False, header=True)

Просмотрим содержимое директории эксперимента:

In [ ]:
!ls diabetes-train-model-experiment

## Создадим и запускаем Эксперимент

In [ ]:
# Create an experiment
experiment = Experiment(workspace=ws, name=experiment_name)

# Create an SKLearn estimator
estimator = SKLearn(source_directory=experiment_dir,
                    entry_script='diabetes-train-model.py',
                    script_params = {'--reg_rate': 0.1},
                    compute_target='local'
                    )


# Run the experiment and wait result
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

See training details:

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

Просмотрим метрики и выходные данные:

In [ ]:
metrics = run.get_metrics()

for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')

for file in run.get_file_names():
    print(file)

## Регистрируем обученную модель

Копируем путь до обученной модели, чтобы зарегистрировать ее.

In [ ]:
model_path = run.get_file_names()[3]
model_path

In [ ]:
run.register_model(model_path=model_path, model_name='diabetes_model',
                   tags={'Dataset':'Diabetes'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

Просмотрим список зарегистрированных моделей:

In [ ]:
for model in Model.list(ws):
    print(f'{model.name} v{model.version}')
    
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print(f'\t {tag_name}: {tag}')
        
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print(f'\t {prop}: {prop_name}')
        
    print('\n')

Проcмотрите зарегистрированную модель на портале Azure ML в разделе: `<your_workspace> > Models > diabetes_model`.